In [1]:
import torch
import gc
torch.cuda.empty_cache()
gc.collect()


0

In [2]:
from setup import setup_src_path
print(setup_src_path())
import data.processed as processed
import config.config as config
import utils.setup as setup
import utils.functions as fn
from importlib import reload



['/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/mlm', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages', '/home/guest/Desktop/projects/third-experiments/domain_adaptation_project/modules']


2024-06-05 11:43:25.981533: I tensorflow/core/util/port.cc:110] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-06-05 11:43:26.021393: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-06-05 11:43:26.700573: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
from datasets import load_from_disk,concatenate_datasets


adapter_name="mlm_invpr_union_G"
dataset = load_from_disk(f"{config.Config.DATASETS_SAVE_PATH}/datasets")



In [4]:
reload(config)

    
filtered_target = dataset['train'].filter(lambda example: example['genre'] == "government").shuffle(seed=42)
#filtered_target = shuffled_filtered_target.train_test_split(test_size=0.1)


filtered_test_target = dataset['validation_matched'].filter(lambda example: example['genre'] == "government")
train_target = filtered_target
test_target = filtered_test_target

 

In [5]:
from adapters import AutoAdapterModel,init
from tqdm import tqdm

from transformers import AutoConfig, AutoModelForMaskedLM

mdlcfg = AutoConfig.from_pretrained(
    config.Config.MODEL_NAME,
 
)
model = AutoModelForMaskedLM.from_pretrained(
    config.Config.MODEL_NAME,
)
init(model)
reload(fn)
fn.print_trainable_parameters(model)

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertFor

trainable params: 109514298 || all params: 109514298 || trainable%: 100.0


In [6]:
from adapters import ConfigUnion, LoRAConfig, PrefixTuningConfig, SeqBnConfig,SeqBnInvConfig,AdapterConfig,LoRAConfig

union_config = ConfigUnion(
    PrefixTuningConfig(prefix_length=10),
    #LoRAConfig(r=8, alpha=16),

    SeqBnInvConfig(reduction_factor=2),
)
model.add_adapter(adapter_name, config=union_config)

model.train_adapter([adapter_name])
model.active_adapters = adapter_name
print(model.adapter_summary())
fn.print_trainable_parameters(model)

Name                     Architecture         #Param      %Param  Active   Train
--------------------------------------------------------------------------------
mlm_invpr_union_G        union            17,244,800      15.837       1       1
--------------------------------------------------------------------------------
Full model                               108,891,648     100.000               0
trainable params: 17836928 || all params: 126759098 || trainable%: 14.071516980974415


In [7]:
from transformers import AutoTokenizer

reload(processed)
tokenizer = AutoTokenizer.from_pretrained(config.Config.TOKENIZER_NAME)

tokenized_dataset= processed.tokenize_dataset(train_target,tokenizer)
tokenized_dataset

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids'],
    num_rows: 77350
})

In [8]:
chunk_size = 128
# Slicing produces a list of lists for each feature
tokenized_samples = tokenized_dataset[444:470]

results = fn.group_texts(tokenized_samples, chunk_size)
for chunk in results["labels"]:
    print(f"'>>> Chunk length: {len(chunk)}'")

'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'
'>>> Chunk length: 128'


In [9]:
# we group texts and chunk them
lm_datasets = tokenized_dataset.map(fn.group_texts, batched=True,fn_kwargs={'chunk_size': chunk_size})
lm_datasets

Dataset({
    features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
    num_rows: 25770
})

In [10]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm_probability=0.15)
samples = [lm_datasets[i] for i in range(2)]
for sample in samples:
    _ = sample.pop("word_ids")
dd=data_collator(samples)
for chunk in dd["input_ids"]:
    print(f"\n'>>> {tokenizer.decode(chunk)}'")


'>>> [CLS] some investments in stewardship land, for example national parks, will [MASK] vicki by both 1 ) the number [MASK] acres used as a park or an historic site in the steward [MASK] landש, and 2 ) by the number of physical units identified as national parks in [MASK] [MASK] [MASK] category [MASK] [SEP] the number of acres used and the number [MASK] [MASK] units [MASK] on stewardship [MASK] are both reported. [SEP] [CLS] the benefits to be gained in air quality were calculated by epa [MASK] the first six years [MASK] including the interim program ) to be reductions of 125 [MASK] 000 tons of confederatecarbons, 2 [MASK] 388 [MASK] [MASK] tons of carbon monoxide'

'>>> and 450, 000 tons of nitrogen [MASK]. [SEP] epa estimate that, in the first six years, there [MASK] be 125 [MASK] 000 fewer tons of hydrocarbons. [SEP] [CLS] 6 [MASK]. [SEP] 10 trillion [SEP] [CLS] professional training organizations to [MASK] on - campus training at very [SEP] there will now be many places to [MASK]

In [11]:

downsampled_dataset = lm_datasets.train_test_split(
    test_size=0.1, seed=42
)
downsampled_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 23193
    })
    test: Dataset({
        features: ['input_ids', 'token_type_ids', 'attention_mask', 'word_ids', 'labels'],
        num_rows: 2577
    })
})

In [12]:

reload(fn)
trainer = fn.train_mlm_model(model,adapter_name,data_collator,tokenizer, downsampled_dataset['train'],downsampled_dataset['test'])

/home/guest/.cache/pypoetry/virtualenvs/third-experments-xuKQSur9-py3.8/lib/python3.8/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [13]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/81 [00:00<?, ?it/s]

>>> Perplexity: 16.46


In [14]:
trainer.train()

  0%|          | 0/14500 [00:00<?, ?it/s]

{'loss': 2.2971, 'grad_norm': 1.9722827672958374, 'learning_rate': 9.84e-05, 'epoch': 1.0}


  0%|          | 0/81 [00:00<?, ?it/s]

{'eval_loss': 1.9781514406204224, 'eval_runtime': 21.5089, 'eval_samples_per_second': 119.811, 'eval_steps_per_second': 3.766, 'epoch': 1.0}
{'loss': 2.0218, 'grad_norm': 1.7863494157791138, 'learning_rate': 9.322857142857144e-05, 'epoch': 2.0}


  0%|          | 0/81 [00:00<?, ?it/s]

...

  0%|          | 0/81 [00:00<?, ?it/s]

{'eval_loss': 1.610639214515686, 'eval_runtime': 8.6039, 'eval_samples_per_second': 299.515, 'eval_steps_per_second': 9.414, 'epoch': 20.0}


Overwriting existing adapter 'mlm_invpr_union_G'.


{'train_runtime': 7164.3643, 'train_samples_per_second': 64.745, 'train_steps_per_second': 2.024, 'train_loss': 1.7722979581109408, 'epoch': 20.0}


TrainOutput(global_step=14500, training_loss=1.7722979581109408, metrics={'train_runtime': 7164.3643, 'train_samples_per_second': 64.745, 'train_steps_per_second': 2.024, 'train_loss': 1.7722979581109408, 'epoch': 20.0})

In [15]:
import math

eval_results = trainer.evaluate()

print(f">>> Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

  0%|          | 0/81 [00:00<?, ?it/s]

>>> Perplexity: 4.84


In [16]:
import config.config as config

trainer.model.save_adapter(f"{config.Config.ADAPTER_SAVE_PATH}/{adapter_name}", adapter_name)


: 

#### This is a report of mlm for goverment domain using whole genre as target and then splitt after tokenization with seed 42. I used computer metrix for preplexity and early stopping 
The results before:
>>> Perplexity: 16.83

The results after:
>>> Perplexity: 4.80